In [ ]:
import numpy as np
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import Constraint
import logging

In [ ]:
%matplotlib inline

logging.basicConfig(level="INFO")

In [ ]:
prod=pd.read_csv("RESMVLVPROD_20201001_20201031 (1).csv")
# Convert the date and hour columns to strings
prod['DATE'] = prod['DATE'].astype(str)
prod['HOUR'] = prod['HOUR'].astype(str)
# Pad the hour values with leading zeros if necessary
prod['HOUR'] = prod['HOUR'].str.zfill(2)
# Adjust hour '24' to '00' and increment the date by one DATE
prod.loc[prod['HOUR'] == '24', 'HOUR'] = '00'
prod['DATE'] = pd.to_datetime(prod['DATE'], format='%Y%m%d')
prod.loc[prod['HOUR'] == '00', 'DATE'] += pd.DateOffset(days=1)
prod['DATE'] = prod['DATE'].dt.strftime('%Y-%m-%d')
prod['datetime'] = prod['DATE'] + ' ' + prod['HOUR'] + ':00:00'
prod['datetime'] = pd.to_datetime(prod['datetime'])
prod.set_index(['datetime'],inplace=True)
start_date = pd.Timestamp('2020-10-25 00:00:00')
end_date = pd.Timestamp('2020-10-31 23:00:00')
prod=prod[start_date:end_date]
prod['ENERGY(MWh)']=prod['ENERGY(MWh)']/2000
prod


In [ ]:
cons = pd.read_csv("HVCUSTCONS_20201001_20201031 (1).csv")
#20201025,25,724.9618378,37.28140312

# Convert the date and hour columns to strings
cons['date'] = cons['date'].astype(str)
cons['hour'] = cons['hour'].astype(str)

# Pad the hour values with leading zeros if necessary
cons['hour'] = cons['hour'].str.zfill(2)

# Adjust hour '24' to '00' and increment the date by one day
cons.loc[cons['hour'] == '24', 'hour'] = '00'
cons['date'] = pd.to_datetime(cons['date'], format='%Y%m%d')
cons.loc[cons['hour'] == '00', 'date'] += pd.DateOffset(days=1)
cons['date'] = cons['date'].dt.strftime('%Y-%m-%d')

# Merge the date and hour columns
cons['datetime'] = cons['date'] + ' ' + cons['hour'] + ':00:00'

# Optionally, convert the 'datetime' column to a datetime data type
cons['datetime'] = pd.to_datetime(cons['datetime'])
cons.set_index(['datetime'],inplace=True)
start_date = pd.Timestamp('2020-10-25 00:00:00')
end_date = pd.Timestamp('2020-10-31 23:00:00')
cons=cons[start_date:end_date]
cons['energy(MWh)']=cons['energy(MWh)']/1000
#cons

In [ ]:
network = pypsa.Network()
network.set_snapshots(prod.index)
#network.snapshots

In [ ]:
network.add("Bus", "Coal", carrier='AC')
network.add("Carrier", "coal", co2_emissions=0.2)

#network.add("Bus", "Solar", carrier="AC")
#network.add("Carrier", "Solar")

#network.add("Bus","Load Bus")

#network.add("Line","Line1",bus0="Coal",bus1="Load Bus",r=0.01,x=0.1)
#network.add("Line","Line2",bus0="Solar",bus1="Load Bus",r=0.01,x=0.1)

network.add("GlobalConstraint", "co2_limit", sense="<=", constant=0.0)

network.add(
    "Generator",
    "Coal Gen",
    bus="Coal",
    carrier="coal",
    p_nom=1, #MW 
    p_nom_extendable=False,
    #committable=True,
    p_set=1,
    #p_max_pu=1,
    p_min_pu=0,
    control="PQ",
    capital_cost=100, #currency/MW
    #marginal_cost=  #currency/MWh 
    )

network.add(
    "Generator",
    "PV Park",
    bus="Coal",
    #carrier="Solar",
    control="PQ",
    p_nom=1, #MW 
    p_nom_extendable=True,
    p_set=np.array(prod['ENERGY(MWh)']),
    p_max_pu=np.array(prod['ENERGY(MWh)']),
    p_min_pu=np.array(prod['ENERGY(MWh)']),
    capital_cost=300, #currency/MW
    #marginal_cost=  #currency/MWh
    #p=max_pu * p_nom   
    )

network.add("Load", "load 1", bus="Coal", p_set=np.array(cons['energy(MWh)']))
#network.add("Store", "store", bus="Solar", carrier="Solar", e_initial=True, e_nom_extendable=True)


In [ ]:
network.lpf()
#network.optimize()

In [ ]:
network.iplot()

In [ ]:
network.buses

In [ ]:
network.loads_t.p

In [ ]:
network.generators_t.p

In [ ]:
network.generators.loc["Coal Gen"]

In [ ]:
network.generators_t.p.plot(ylabel='ΜW')
network.loads_t.p.plot(ylabel='ΜW')